In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
import pyspark.sql.functions as F 
#from pyspark.sql.functions import col, min, max, expr

In [3]:
conf = SparkConf() \
    .setAppName("pleio_vs_purif_boxplot")\

# Create a SparkContext with the specified configurations
if 'spark' in locals() and spark!=None:
    spark.stop()

sc = SparkContext(conf=conf)

# Create a SparkSession from the SparkContext
spark = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/06 12:58:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/06 12:58:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
#loading in all autosomes
#Skipping sex chromosomes, see readme
df = spark.read \
    .option("comment", "#") \
    .option("delimiter", ",") \
    .csv("/gpfs/gibbs/pi/reilly/VariantEffects/scripts/noon_data/2.filter/*.csv/*.csv.gz", header=True)

24/02/06 12:59:05 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [5]:
cell_types=["K562","SKNSH","HepG2"]
for cell_type in cell_types:
    df = df.withColumn(f"emVar_{cell_type}", 
                           (F.abs(F.col(f"{cell_type}__skew")) >= 0.5) & 
                           (F.greatest(F.col(f"{cell_type}__ref"), F.col(f"{cell_type}__alt")) >= 1.0))

#next, we count the number of cell-types each variant is an emvar in to compute the pleitropy. 
df = df.withColumn("pleio", F.col("emVar_K562").cast("int") + F.col("emVar_SKNSH").cast("int") + F.col("emVar_HepG2").cast("int"))

In [6]:
df=df.withColumn("AF",F.col("AF").cast("float"))

Todo: rewrite to be cleaner & work for more subdivisions: espc. cell-type. 

In [16]:
distinct_categories = [0,1,2,3]
quantile_probs = [0.25, 0.5, 0.75]  # For quartiles
quantile_reliability = 0.05  # Adjust based on your needs

def gen_box_quantiles(df):
    
    quantiles_by_category=[]
    
    
    
    for category_val in distinct_categories:
        sub_df = df.where(F.col('pleio') == category_val)
        quantiles = sub_df.stat.approxQuantile('AF', quantile_probs, quantile_reliability)
        quantiles_by_category.append((category_val, *quantiles))
    
    return quantiles_by_category

    
#quantiles_by_category=gen_box_quantiles(df)
#print(quantiles_by_category)

Let's get the min & max

In [12]:
min_max = (
    df.groupBy('pleio')
    .agg(
        F.min('AF').alias('min'),
        F.max('AF').alias('max')
    )
)

In [15]:
min_max.collect()

[Row(pleio=None, min=6.565469902852783e-06, max=1.0),
 Row(pleio=1, min=6.565469902852783e-06, max=1.0),
 Row(pleio=3, min=6.565469902852783e-06, max=1.0),
 Row(pleio=2, min=6.565469902852783e-06, max=1.0),
 Row(pleio=0, min=6.565469902852783e-06, max=1.0)]

Collect the ALL statistics

In [17]:
all_quantiles = df.stat.approxQuantile('AF', quantile_probs, quantile_reliability)

In [18]:
all_min_max=df.agg(F.min('AF').alias('min'),F.max('AF').alias('max'))

Dump to disc so I don't have to run all of the above cells every time.

In [ ]:
import pickle
data_to_dump={"all_quantiles":all_quantiles,"all_min_max":all_min_max.toPandas(),"quantiles_by_category":quantiles_by_category.toPandas(),"min_max":min_max.toPandas()}

filename="temp_dump.pkl"

with open(filename, 'wb') as file:
    pickle.dump(data_to_dump, file)

get stats for difference of means ttest & plotting
- mean
- variance
- standard deviation
- n

In [ ]:
output={tropy:[],region:[],n:[],mean:[],stdev:[]}

for tropy in [0,1,2,"ALL"]:
    for region in ["is_in_PLS","is_in_dELS","is_in_pELS","ALL"]:
        output
        
        #subset=df
        #pass
output